# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119624e+02     1.371171e+00
 * time: 0.4870460033416748
     1     1.122311e+01     8.764239e-01
 * time: 1.0943191051483154
     2    -1.175265e+01     9.272386e-01
 * time: 1.15818190574646
     3    -3.407199e+01     7.505241e-01
 * time: 1.2800359725952148
     4    -4.745053e+01     5.694316e-01
 * time: 1.3615350723266602
     5    -5.691136e+01     2.255406e-01
 * time: 1.4542279243469238
     6    -5.975202e+01     1.924874e-01
 * time: 1.5405960083007812
     7    -6.089204e+01     7.525978e-02
 * time: 1.5955500602722168
     8    -6.135514e+01     3.916752e-02
 * time: 1.6501679420471191
     9    -6.161536e+01     3.903533e-02
 * time: 1.706408977508545
    10    -6.178912e+01     2.833005e-02
 * time: 1.7622160911560059
    11    -6.194418e+01     2.213022e-02
 * time: 1.817647933959961
    12    -6.204867e+01     1.679010e-02
 * time: 1.8920629024505615
    13    -6.209410e+01     1.602960e-02
 * time: 1.94710993766784

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671076
    AtomicLocal         -18.8557697
    AtomicNonlocal      14.8522653
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485379 
    Xc                  -19.3336823

    total               -62.261666458994